In [39]:
## Tropospheric delay correction using GACOS from New Castle U
## for geocoded dataset
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from pysar.objects import timeseries
from pysar.utils import ptime, readfile, utils as ut
import h5py
#/media/lei/disk1/loading/tehri/20190427/SenDT63
#/media/lei/disk2/ErtaAle/20190605/SenAT14
ztd_dir = os.path.expanduser('/media/lei/disk2/ErtaAle/20190606/SenDT79/WEATHER/GACOS')
proj_dir = os.path.expanduser('/media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR')
work_dir = '/media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR'

out_file = os.path.join(proj_dir, 'INPUTS/GACOS.h5')
ts_file = os.path.join(proj_dir, 'timeseries.h5')
geom_file = os.path.join(proj_dir, 'INPUTS/geometryRadar.h5')
mask_file = os.path.join(proj_dir, 'lava_d.h5')

In [40]:
def get_geo_delay(fztd, geo_box, atr, inc_angle):
    meta = readfile.read_attribute(fztd)
    coord = ut.coordinate(meta)
    box = coord.box_geo2pixel(geo_box)
    phs = readfile.read(fztd, box=box)[0]
    phs[phs == 0] = np.nan

    # interpolation
    out_shape = (int(atr['LENGTH']), int(atr['WIDTH']))
    phs = resize(phs, out_shape, order=1, mode='edge', anti_aliasing=True)

    phs /= np.cos(inc_angle * np.pi / 180)
    phs -= phs[int(atr['REF_Y']), int(atr['REF_X'])]
    phs *= -1
    return phs

In [41]:
# 1. get time-series file info
obj = timeseries(ts_file)
obj.open()
atr = readfile.read_attribute(ts_file)

# get box_box
pixel_box = (0, 0, int(atr['WIDTH']), int(atr['LENGTH']))
geo_box = ut.coordinate(atr).box_pixel2geo(pixel_box)

# get inc_angle
inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]

# get grib file list
fztd_list = [os.path.join(ztd_dir, '{}.ztd'.format(i)) for i in obj.dateList]

open timeseries file: timeseries.h5


In [42]:
# calculate deley time-series
num_date = obj.numDate
tropo = np.zeros((num_date, obj.length, obj.width), dtype=np.float32)
prog_bar = ptime.progressBar(maxValue=num_date)
# if os.path.isfile(mask_file):
#         h5mask=h5py.File(mask_file,'r')
#         mask = h5mask['mask'][:,:]
#         print('mask existed')
# else:
#     mask = np.ones((obj.length, obj.width))
for i in range(num_date):
    fztd = fztd_list[i]
    tropo[i, :, :] = get_geo_delay(fztd, geo_box, atr, inc_angle)
#     tropo[i,:,:] = mask*tropo[i,:,:]
    prog_bar.update(i+1, suffix=os.path.basename(fztd))
prog_bar.close()
tropo -= np.tile(tropo[0, :, :], (num_date, 1, 1))

# write deley to h5 file
obj_out = timeseries(out_file)
obj_out.write2hdf5(data=tropo,
                   dates=obj.dateList,
                   metadata=atr,
                   refFile=ts_file)

[==================================================] 20190506.ztd    2s /     0s 
create timeseries HDF5 file: /media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR/INPUTS/GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (67, 650, 480) with compression=None
create dataset /dates      of |S8        in size of (67,)
create dataset /bperp      of float32    in size of (67,)
finished writing to /media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR/INPUTS/GACOS.h5


'/media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR/INPUTS/GACOS.h5'

In [47]:
# correct time-series
os.chdir(work_dir)
print('Go to directory', work_dir)

!diff.py timeseries.h5 INPUTS/GACOS.h5 -o timeseries_GACOS.h5
!remove_ramp.py timeseries_GACOS.h5 -m maskTempCoh.h5 -s linear
!dem_error.py timeseries_GACOS.h5 -t pysarApp_template.txt
!dem_error.py timeseries_GACOS_ramp.h5 -t pysarApp_template.txt

!geocode.py timeseries_GACOS_demErr.h5 -t pysarApp_template.txt
!geocode.py timeseries_GACOS_ramp_demErr.h5 -t pysarApp_template.txt

Go to directory /media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR
timeseries.h5 - ['INPUTS/GACOS.h5'] --> timeseries_GACOS.h5
input files are: timeseries and timeseries
open timeseries file: timeseries.h5
open timeseries file: GACOS.h5
reading timeseries data from file: timeseries.h5 ...
create timeseries HDF5 file: timeseries_GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (67, 650, 480) with compression=None
create dataset /dates      of |S8        in size of (67,)
create dataset /bperp      of float32    in size of (67,)
finished writing to timeseries_GACOS.h5
remove linear ramp from file: timeseries_GACOS.h5
read mask file: maskTempCoh.h5
reading data ...
estimating phase ramp ...
create timeseries HDF5 file: timeseries_GACOS_ramp.h5 with w mode
create dataset /timeseries of float32    in size of (67, 650, 480) with compression=None
create dataset /dates      of |S8        in size of (67,)
create dataset /bperp      of float32    in size of (67,)
finished wri

In [48]:
!view.py timeseries_GACOS_demErr.h5 -v -25 15 --nodisplay
!view.py timeseries_ECMWF_demErr.h5 -v -25 15 --nodisplay

run view.py in PySAR release version v1.1.0-15, release date 2019-06-12
input file is timeseries file: /media/lei/disk2/ErtaAle/20190606/SenDT79/PYSAR/timeseries_GACOS_demErr.h5 in float32 format
file size in y/x: (650, 480)
num of datasets in file timeseries_GACOS_demErr.h5: 67
datasets to exclude (0):
[]
datasets to display (67):
['timeseries-20170209', 'timeseries-20170221', 'timeseries-20170305', 'timeseries-20170317', 'timeseries-20170329', 'timeseries-20170410', 'timeseries-20170422', 'timeseries-20170504', 'timeseries-20170516', 'timeseries-20170528', 'timeseries-20170609', 'timeseries-20170621', 'timeseries-20170703', 'timeseries-20170715', 'timeseries-20170727', 'timeseries-20170808', 'timeseries-20170820', 'timeseries-20170901', 'timeseries-20170913', 'timeseries-20170925', 'timeseries-20171007', 'timeseries-20171019', 'timeseries-20171031', 'timeseries-20171112', 'timeseries-20171124', 'timeseries-20171206', 'timeseries-20171218', 'timeseries-20171230', 'timeseries-20180111'